<a href="https://colab.research.google.com/github/robertccruz13-ship-it/sds-510-robert-cruz/blob/main/module_5_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()
#

Saving jeopardy.json to jeopardy.json


In [3]:
import pandas as pd
import numpy as np

df = pd.read_json('jeopardy.json')

df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216930 entries, 0 to 216929
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   category     216930 non-null  object
 1   air_date     216930 non-null  object
 2   question     216930 non-null  object
 3   value        213296 non-null  object
 4   answer       216930 non-null  object
 5   round        216930 non-null  object
 6   show_number  216930 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 11.6+ MB


In [7]:
df = df[['question', 'value']].dropna()

df['Value_clean'] = (
    df['value']
    .astype(str)
    .str.replace('[$,]', '', regex=True)
    .replace('None', np.nan)
)

df = df.dropna(subset=['Value_clean'])
df['Value_clean'] = df['Value_clean'].astype(int)

In [10]:
threshold = 800
df['label'] = (df['Value_clean'] == threshold).astype(int)

df['label'].value_counts()

,count
label,
0,181436
1,31860


In [13]:
from sklearn.model_selection import train_test_split

X = df['question']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

clf = MultinomialNB()
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Accuracy: 0.8385607126113456
              precision    recall  f1-score   support

           0       0.85      0.98      0.91     36288
           1       0.15      0.02      0.03      6372

    accuracy                           0.84     42660
   macro avg       0.50      0.50      0.47     42660
weighted avg       0.75      0.84      0.78     42660

